In [1]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
from pandas.io.json import json_normalize
import nltk
import re
from rouge import Rouge 
from collections import Counter
from geotext import GeoText
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
import string
from gensim.summarization.summarizer import summarize

from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer #We're choosing Lexrank, other algorithms are also built in
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

import articleDateExtractor


global str

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package maxen

In [2]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
# nltk.internals.config_java("C:/Program Files/Java/jdk1.8.0_181/bin")

import os
java_path = "C:/Program Files/Java/jdk1.8.0_181/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('C:/Users/anike/Downloads/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz','C:/Users/anike/Downloads/stanford-ner-2018-10-16/stanford-ner-3.9.2.jar', encoding='utf-8')


# Classifier 

In [3]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
import spacy
import re
import csv
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score

C:\Users\anike\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\anike\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
lemma = nltk.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess(rawText):
    cleanText = re.sub(r'[^\w\s]','',rawText.lower())
    tokens = word_tokenize(cleanText)
    tokens = [token for token in tokens if not token in stop_words]
    tokens = [lemma.lemmatize(token) for token in tokens ]
    processedText = (" ".join(tokens))
    return processedText

# Importing dataset

In [5]:
data = open(r'C:\Users\anike\indonesiaEventCorpus_2010_2019.csv')
# data = open(r'C:\Users\anike\indiaCorpus_2010_2019.csv')

target, notes = [], []
reader = csv.reader(data, delimiter=",")
next(reader, None)
for i, line in enumerate(reader):
    target.append(line[1])
    rawText = line[0]
    notes.append(preprocess(rawText))

dataSet = pd.DataFrame()
dataSet['notes'] = notes
dataSet['target'] = target
dataSet = dataSet.sample(frac=1).reset_index(drop=True)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(dataSet['notes'], dataSet['target'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
vector.fit(dataSet['notes'])
train_x =  vector.transform(train_x)
valid_x =  vector.transform(valid_x)

# Naieve Bayes Event Classifier

In [6]:

clf = MultinomialNB().fit(train_x, train_y)
predicted = clf.predict(valid_x)
print(accuracy_score(valid_y,predicted))

0.72


# SVM Event Classifier

In [7]:
from sklearn import svm
SVM = svm.LinearSVC()
SVM.fit(train_x, train_y)
predict = SVM.predict(valid_x)
accuracy_score(valid_y,predict)

0.8514285714285714

# Random Forest Event Classifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(train_x, train_y)
predict = rf.predict(valid_x)
accuracy_score(valid_y,predict)

0.7771428571428571

In [19]:
LIMIT = 4
data = {}
data['newspapers'] = {}
articlesData ={}
collections = {
    'total' : []
}


In [22]:
# with open('newspaperLinks.json') as file:
with open('newspaperLinks_Indonesia.json') as file:
    sources = json.load(file)
sources

{'World News Report': {'rss': 'https://www.einnews.com/rss/T2tHVmwcBIBT0pom',
  'link': 'https://www.einnews.com/rss/T2tHVmwcBIBT0pom'}}

In [20]:
def summarizeText(text):
    string=text
    parser = PlaintextParser.from_string(string, Tokenizer("english"))

    summarizer = LexRankSummarizer()
    summary_lex = summarizer(parser.document, 1) #Summarize the document with 5 sentences
    summary_1=''
    for line in summary_lex:
        #summary_1+=line
        #print(line)
        summary_1+=str(line)
#     print(summary_1)

#     lsa_summarizer = LsaSummarizer()
#     summary_lsa =lsa_summarizer(parser.document,3)
#     summary_2=''
#     print("\n")
#     for line in summary_lsa:
#         summary_2+=str(line)
#     print(summary_2)

#     luhn_summarizer = LuhnSummarizer() 
#     luhn_summarizer.stop_words = ("I",  "me", "is", "am", "the","than", "that", "this","an","you", "are")
#     summary_3=''
#     print("\n")
#     for line in luhn_summarizer(parser.document, 3):
#         summary_3+=str(line)
#     print(summary_3)
    return summary_1

def getLocation(ents, text):
    loc = ""
#     locList = []
    items = [x.text for x in ents if x.label_ in 'GPE' or x.label_ in 'LOC']
    loc_ = [Counter(items).most_common(1)]
    places = GeoText(text)
    topPlace = [Counter(places.cities).most_common(1)]
    
    places = (places.cities)
#     text = re.sub(r'[^\w\s]','',text)
#     tokenized_text = word_tokenize(text)
#     classified_text = st.tag(tokenized_text)
#     stanfordLocations =[]
#     for x in classified_text:
#         if(x[1]=="LOCATION"):
#             stanfordLocations.append(x[0])
#     topPlace = [Counter(stanfordLocations).most_common(1)]
#     locList = [i for i in stanfordLocations if i in items]
    locList = [i for i in places if i in items]
    total = items+locList 
    location_ = [Counter(total).most_common(1)]
    if len(location_) == 0:
        if(len(loc_))!=0:
            loc = loc_[0][0][0]
        elif (len(topPlace)!=0):
            loc = topPlace[0]
        else:
            loc = ""
    else:
        if(len(location_))!=0:
            if(len(location_[0]))!=0:
                if(len(location_[0][0]))!=0:
                    loc = location_[0][0][0]
        else:
            loc = "unknown"
    return loc

def getEventType(tokens):
#     eventTypes ={
#     'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood'],
#     'riotsProtest':['protest','riot']
#     }
    eventType = "other"
    for event, types in eventTypes.items():
        tag = [token for token in tokens if any(typ in token for typ in types)]
        if len(tag) != 0:
            eventType = event
            break
    return eventType

# News Scraper and Classifier

In [23]:
collections = {
    'total' : []
}
from time import mktime
from datetime import datetime
from lexrank import STOPWORDS, LexRank

event_type = ""
data = {}
data['newspapers'] = {}
articlesData ={}
collectedNews = {}
politics= ["politicize", "political affairs","election","polling", "administration", "Indonesia Politics", "politicians", "elections", "votes", "voters","vote","voting"]
violence = ["violence", "criminal violence", "criminal", "crime", "criminal party", "election violence", "electoral violence","mob killing", "vigilante", "riots", "killing" ]
parties = ['democratic Party', 'golkar', 'indonesian democratic party of struggle ', 'pdpi', 'pdp-i', 'prosperous justice party', 'pks', 'national mandate party', 'pan', 'united development party', 'ppp', 'national awakening party', 'pkb','great indonesia movement party','gerindra','peoples conscience party','hanura','nasdem party','indonesian solidarity party','psi','berkarya Party']
events = ['arrest','arrested','violence against civilians','rioting','protesting','riots','protest','battles','violence','remote violence','assault','attack','bloodshed','brutality','clash','confusion','cruelty','disorder','disturbance','fighting','rampage','struggle','terrorism','abandon','acuteness','bestiality','blowup','coercion','compulsion','constraint','destructiveness','duress','ferocity','fervor','fierceness','flap','frenzy','fury','fuss','harshness','murderousness','onslaught','passion','power','roughness','ruckus','rumble','savagery','scam','severity','sharpness','storm','storminess','tumult','turbulence','uproar','vehemence','wildness','brute','force','foul','play','raging','anarchy','brawl','disturbance','lawlessness','protest','storm','strife','trouble','turbulence','turmoil','uproar','anarchism','burst','commotion','confusion','distemper','flap','fray','free-for-all','fuss','hassle','misrule','mix-up','quarrel','racket','row','ruckus','ruction','rumble','rumpus','run-in','scene','shivaree','shower','snarl','stir','to-do','tumult','brannigan','mob','violence','street','fighting','wingding','action','assault','attack','bloodshed','bombing','campaign','clash','combat','conflict','crusade','encounter','fighting','hostility','skirmish','strife','struggle','war','warfare','barrage','brush','carnage','contention','engagement','fray','havoc','onset','onslaught','press','ravage','scrimmage','sortie','blitzkreig']

eventTypes ={
'riots':['riot'],
'protests':['protest','march'], 
'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood']
}



count = 1

for source, link in sources.items():
    print(source)
    count = 1
    if 'rss' in link:
        parsedFeed = fp.parse(link['rss'])
        collected = {
            'rss': link['rss'],
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.entries:
            
            try:

                articlesData ={}
                articles = Article(feed['links'][0]['href'])
                articles.download()
                articles.parse()
                articles.nlp()
                strg = re.sub(r'[^\w\s]','',articles.text)
                articleTokens = nlp(strg)

                tokens =[]
                for token in articleTokens:
                    tokens.append(token.text.lower())

                txt = str(articles.text).split()

                keyWords = articles.keywords

                eventType = "other"
                if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                    print(articles.title)
                    articlesData['title'] = articles.title

                    article = nlp(articles.text)
                    ###Summary
                    sentences = [x for x in article.sents]
                    sentences = list( map(str, sentences) )
                    lxr = LexRank(articles.text)
                    lxr.get_summary(sentences)
                    summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                    summary = (" ".join(summary))
                    
                    rawSummary = [preprocess(summary)]
                    textVector = vector.transform(rawSummary)
                    prediction = rf.predict(textVector)
                    print(encoder.inverse_transform(prediction))
                    articlesData['event'] = encoder.inverse_transform(prediction[0])

                    dates = articleDateExtractor.extractArticlePublishedDate(feed['links'][0]['href'])
                    localFormat = dates.strftime("%m/%d/%Y")
                    summaryDate = dates.strftime("On %d %b,")
                    articlesData['eventDate'] = localFormat
                    articlesData['location'] = getLocation(articleTokens.ents, articles.text)
                    orgs = set(keyWords).intersection(set(parties))

                    if len(orgs) == 0:
                        if(articlesData['event']=="Riots"):
                            taggedOrg = "Rioters"
                        elif(articlesData['event']=="Protests"):
                            taggedOrg = "Protestors"
                        else:
                            taggedOrg = "other"
                    else:
                        taggedOrg=""
                        for org in orgs:
                            taggedOrg+=org+", "
                        taggedOrg=taggedOrg[:-2]

                    articlesData['partiesInvolved'] = taggedOrg
                    articlesData['source'] = source
                    totalSummary = []
                    totalSummary.append(summaryDate)
                    totalSummary.append(summary)
                    extractiveSummary  = (" ".join(totalSummary))
                    articlesData['summary'] = extractiveSummary
                    articlesData['link'] = (feed['links'][0]['href'])
                    collected['article'].append(articlesData)
#                 else:
#                     break
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)
    else:
        parsedFeed = newspaper.build(link['link'], memoize_articles = False)
        collected = {
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.articles:
#             if count > 1000:
#                 break
            try:
                feed.download()
                feed.parse()
                feed.nlp()
                if feed.publish_date is not None:
                    articlesData ={}
                    
                    strg = re.sub(r'[^\w\s]','',feed.text)
                    articleTokens = nlp(strg)
                    tokens =[]
                    for token in articleTokens:
                        tokens.append(token.text.lower())

                        txt = str(feed.text).split()
                        for i in txt:
                            for j in events:
                                if i==j:
                                    event_type = j
                                    break
                                else:
                                    event_type = "other"
                    keyWords = feed.keywords
                    if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                        
                        articlesData['title'] = feed.title
                        article = nlp(feed.text)
                        ###Summary
                        sentences = [x for x in article.sents]
                        sentences = list( map(str, sentences) )
                        lxr = LexRank(feed.text)
                        lxr.get_summary(sentences)
                        summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                        summary = (" ".join(summary))

                        rawSummary = [preprocess(summary)]
                        textVector = vector.transform(rawSummary)
                        prediction = rf.predict(textVector)
                        print(encoder.inverse_transform(prediction))
                        articlesData['event'] = encoder.inverse_transform(prediction[0])
                        
#                         articlesData['event'] = getEventType(tokens)
                        articlesData['eventDate'] = feed.publish_date.strftime('%m/%d/%Y')
                        articlesData['location'] = getLocation(articleTokens.ents,feed.text)
                        
                        orgs = set(keyWords).intersection(set(parties))
                        if len(orgs) == 0:
                            if(articlesData['event']=="Riots"):
                                taggedOrg = "Rioters"
                            elif(articlesData['event']=="Protests"):
                                taggedOrg = "Protestors"
                            else:
                                taggedOrg = "other"
                        else:
                            taggedOrg=""
                            for org in orgs:
                                taggedOrg+=org+", "
                            taggedOrg=taggedOrg[:-2]

                        articlesData['partiesInvolved'] = taggedOrg
                        articlesData['source'] = source
                        totalSummary = []
                        totalSummary.append(feed.publish_date.strftime("On %d %b,"))
                        totalSummary.append(summary)
                        extractiveSummary  = (" ".join(totalSummary))
                        articlesData['summary'] = extractiveSummary
                        articlesData['link'] = link
                        collected['article'].append(articlesData)
#                     else:
#                         break
#                         count +=1
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)

World News Report
The Natural Resource Oligarchy Funding Indonesia’s Election


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['Other']


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Extracting date from https://www.einnews.com/article/484901884/87tTuc0PYCYyE6Df?ref=rss&ecode=T2tHVmwcBIBT0pom
Exception in extractArticlePublishedDate for https://www.einnews.com/article/484901884/87tTuc0PYCYyE6Df?ref=rss&ecode=T2tHVmwcBIBT0pom
()
'NoneType' object has no attribute 'strftime'
Friday's best photos: Indonesia protests and VR headsets


...

Extracting date from https://www.einnews.com/article/484811977/cNa5YttM44kS5tnN?ref=rss&ecode=T2tHVmwcBIBT0pom
ICMI calls on govt to help losing camp overcome disappointment


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['Protests']


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Extracting date from https://www.einnews.com/article/484791521/N3MVFTlAegD5e9D_?ref=rss&ecode=T2tHVmwcBIBT0pom
Article `download()` failed with 404 Client Error: Not Found for url: http://www.newsonjapan.com/2019/05/10/lessons-learnt-from-indonesian-and-malaysian-approaches-to-terrorist-rehabilitation/ on URL https://www.einnews.com/article/484778218/xOcb2vKEQC_0l2B_?ref=rss&ecode=T2tHVmwcBIBT0pom


In [24]:
collections["total"]

[{'rss': 'https://www.einnews.com/rss/T2tHVmwcBIBT0pom',
  'link': 'https://www.einnews.com/rss/T2tHVmwcBIBT0pom',
  'article': [{'title': "Friday's best photos: Indonesia protests and VR headsets",
    'event': 'Protests',
    'eventDate': '05/10/2019',
    'location': 'London',
    'partiesInvolved': 'Protestors',
    'source': 'World News Report',
    'summary': 'On 10 May, London, England Guy Verhofstadt, centre, who leads the Alliance of Liberals and Democrats for Europe and is running for re-election as an MEP, stands with the Liberal Democrat leader, Vince Cable, and party canvassers\n\nPhotograph: Matt Dunham/AP',
    'link': 'https://www.einnews.com/article/484834140/67AW7Wz6tx7nKhR3?ref=rss&ecode=T2tHVmwcBIBT0pom'},
   {'title': 'Indonesia election tainted, opposition marchers say, but government warns against treason - World',
    'event': 'Protests',
    'eventDate': '05/10/2019',
    'location': 'Jakarta',
    'partiesInvolved': 'Protestors',
    'source': 'World News Repo

In [25]:
frame = json_normalize(collections['total'])
rows = frame.shape[0]
frame

,article,link,rss
0,[{'title': 'Friday's best photos: Indonesia pr...,https://www.einnews.com/rss/T2tHVmwcBIBT0pom,https://www.einnews.com/rss/T2tHVmwcBIBT0pom


In [26]:
totalDF = pd.DataFrame()
for row in range(0,rows):
    tempDF = pd.DataFrame()
    tempDF = json_normalize(collections['total'][row]['article'])
    totalDF = totalDF.append(tempDF, ignore_index = True) 


# Dataframe that contains scraped news which has been classified and summarized

In [27]:
totalDF

,event,eventDate,link,location,partiesInvolved,source,summary,title
0,Protests,05/10/2019,https://www.einnews.com/article/484834140/67AW...,London,Protestors,World News Report,"On 10 May, London, England Guy Verhofstadt, ce...",Friday's best photos: Indonesia protests and V...
1,Protests,05/10/2019,https://www.einnews.com/article/484820064/6yCY...,Jakarta,Protestors,World News Report,"On 10 May, The protesters, spurred by unoffici...","Indonesia election tainted, opposition marcher..."
2,Protests,05/11/2019,https://www.einnews.com/article/484811977/cNa5...,Islamophobia,Protestors,World News Report,"On 11 May, Yet the Liberals have axed funding ...",Australia’s voters are poised to punish the go...
3,Protests,05/10/2019,https://www.einnews.com/article/484791521/N3MV...,Jakarta,Protestors,World News Report,"On 10 May, An exit poll conducted by the Prabo...",ICMI calls on govt to help losing camp overcom...


In [28]:
import datetime
now = datetime.datetime.now()
totalDF.to_excel("airData/indonesia/testing_"+now.strftime("%m%d%Y_%H%M%S")+".xlsx")